In [1]:
pip install psycopg2-binary

     |████████████████████████████████| 3.0 MB 3.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install unidecode

     |████████████████████████████████| 241 kB 2.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import csv
import os
import zipfile

import psycopg2
import psycopg2.extras
import unidecode
import requests
import re

In [5]:
conn = psycopg2.connect(database="campaign-finance",
                        user="postgres",
                       password="",
                        host="172.16.238.13",
                        port="5432")
c = conn.cursor()

## **Committee Documents**

In [ ]:
directory = os.fsencode("./data/committee_list")
    
for filename in os.listdir(directory):
    full_filename = os.path.join(directory, filename)
    
    with open(full_filename, 'rU') as csv_file: 
        c.copy_expert("COPY committee_list " 
                  "(committeename_url, committeename, " 
                  " sboeid_url, sboeid, " 
                  " status_url, status, " 
                  " candidatename_parententityname_url, candidatename_parententityname) " 
                  "FROM STDIN CSV HEADER", csv_file)

    conn.commit()
print("Done.") 
    

In [ ]:
directory = os.fsencode("./data/committee_doc_list")
    
for filename in os.listdir(directory):
    full_filename = os.path.join(directory, filename)
    print(full_filename)
    with open(full_filename, 'rU') as csv_file: 
        c.copy_expert("COPY committee_doc_list " 
                  "(committee_name, year, " 
                  " doctype, docname, " 
                  " receivedimage, startdate, enddate, " 
                  " image_url, image, "
                  " data_text, data_url) " 
                  "FROM STDIN CSV HEADER", csv_file)

    conn.commit()

print("Done.")
    
    

In [ ]:
print('fixing committee ids ...')
c.execute("update committee_doc_list "
          "set committee_name = split_part(committee_name, '[', 1), "
          " sboe_committee_id = substring(split_part(committee_name, '[', 2), 1, 16) "
          )
print("Done.")

In [ ]:

	set committee_name = split_part(committee_name, '[', 1),
		sboe_committee_id = substring(split_part(committee_name, '[', 2), 1, 16)

In [ ]:
_file = './data/raw_files/transinq_results_20170101_20170630'
contributions_zip_file = _file + '.txt.zip'
contributions_txt_file = _file + '.txt'
contributions_csv_file = _file + '.csv'

In [ ]:
# Create a cleaned up CSV version of file with consistent row lengths.
# Postgres COPY doesn't handle "ragged" files very well
if not os.path.exists(contributions_csv_file):
    print('converting tab-delimited raw file to csv...')
    with open(contributions_txt_file, 'rU', encoding="cp1252") as txt_file, \
            open(contributions_csv_file, 'w') as csv_file:
        csv_writer = csv.writer(csv_file, quoting=csv.QUOTE_ALL)
        for line in txt_file:
            if not all(ord(c) < 128 for c in line):
                line = unidecode.unidecode(line)
            line = re.sub('"','', line)
            row = line.rstrip('\t\r\n').split('\t')
            if len(row) == 24:
                row = row + ['','']
            if len(row) != 24:
                print('skipping bad row (length %s, expected 24):' % len(row))
                print(row)
                continue
            csv_writer.writerow(row)

## **Raw Tables**

These are the files downloaded from the SBOE site transaction search, they only have to be imported once


In [25]:
conn.commit()

In [26]:
print('creating raw_table table...')
c.execute("CREATE TABLE raw_table "
          "(trans_id SERIAL PRIMARY KEY, "
          " name VARCHAR(200) DEFAULT '' NOT NULL, "
          " street_line_1 VARCHAR(200) DEFAULT '' NOT NULL, street_line_2 VARCHAR(200) DEFAULT '' NOT NULL, "
          " city VARCHAR(200) DEFAULT '' NOT NULL, state VARCHAR(15) DEFAULT '' NOT NULL, "
          " zip_code VARCHAR(11) DEFAULT '' NOT NULL, occupation VARCHAR(200) DEFAULT '' NOT NULL, "
          " employer VARCHAR(200) DEFAULT '' NOT NULL, transaction_type VARCHAR(200) DEFAULT '' NOT NULL, "
          " committee_name VARCHAR(200) DEFAULT '' NOT NULL, "
          " committee_sboe_id VARCHAR(200) DEFAULT '' NOT NULL, committee_street_1 VARCHAR(200) DEFAULT '' NOT NULL, "
          " committee_street_2 VARCHAR(200) DEFAULT '' NOT NULL, committee_city VARCHAR(200) DEFAULT '' NOT NULL, "
          " committee_state VARCHAR(200) DEFAULT '' NOT NULL, committee_zip_code VARCHAR(200) DEFAULT '' NOT NULL, "
          " report_name character varying(200) DEFAULT '' NOT NULL, date_occured VARCHAR(200) DEFAULT '' NOT NULL, "
          " account_code VARCHAR(200) DEFAULT '' NOT NULL, amount VARCHAR(200) DEFAULT '' NOT NULL, "
          " form_of_payment VARCHAR(200) DEFAULT '' NOT NULL, purpose VARCHAR(500) DEFAULT '' NOT NULL, "
          " candidate_referendum_name VARCHAR(200) DEFAULT '' NOT NULL, declaration VARCHAR(200) DEFAULT '' NOT NULL "
          " )")

print("Done.")
conn.commit()



creating raw_table table...
Done.


In [9]:
directory = os.fsencode("./data/raw_files")
    
for filename in os.listdir(directory):
    full_filename = os.path.join(directory, filename)
    print(full_filename)
    with open(full_filename, 'rU') as csv_file: 
        c.copy_expert("COPY raw_table " 
                      "(name, street_line_1, street_line_2, city, state, " 
                      " zip_code, occupation, employer, transction_type, " 
                      " committee_name, committee_sboe_id, committee_street_1, " 
                      " committee_street_2, committee_city, committee_state, " 
                      "committee_zip_code, report_name, date_occured, account_code, " 
                      " amount, form_of_payment, purpose, candidate_referendum_name," 
                      " declaration) " 
                      "FROM STDIN CSV HEADER", csv_file)

    conn.commit()

print("Done.")

b'./data/raw_files/transinq_results-20100101-20101231.csv'


<ipython-input-9-50c6f0113282>:6: DeprecationWarning: 'U' mode is deprecated
  with open(full_filename, 'rU') as csv_file:


b'./data/raw_files/transinq_results-20120101-20121231.csv'
b'./data/raw_files/transinq_results_20110101_20111231.csv'
b'./data/raw_files/transinq_results_20130101_20131231.csv'
b'./data/raw_files/transinq_results_20140101_20141231.csv'
b'./data/raw_files/transinq_results_20150101_20151231.csv'
b'./data/raw_files/transinq_results_20160101_20161231.csv'
Done.


## **Transaction Tables**

We are parsing out the information from the raw tables into specific objects 

In [10]:
print('creating accounts table...')
c.execute("CREATE TABLE accounts "
          "(account_id SERIAL PRIMARY KEY, "
          " name VARCHAR(200) DEFAULT '' NOT NULL, "
          " address_1 VARCHAR(200) DEFAULT '' NOT NULL, address_2 VARCHAR(200) DEFAULT '' NOT NULL, "
          " city VARCHAR(200) DEFAULT '' NOT NULL, state VARCHAR(15) DEFAULT '' NOT NULL, "
          " zip VARCHAR(11) DEFAULT '' NOT NULL, occupation VARCHAR(200) DEFAULT '' NOT NULL, "
          " employer VARCHAR(200) DEFAULT '' NOT NULL, "
          " is_donor INT, is_vendor INT, is_person INT, is_organization INT "
          " )")
print("table created, inserting records")
c.execute("INSERT INTO accounts "
          "(name, address_1, "
          " address_2, city, state, zip, occupation, employer) "
          "SELECT DISTINCT "
          "COALESCE(UPPER(TRIM(name)), ''), "
          "COALESCE(UPPER(TRIM(street_line_1)), ''), COALESCE(UPPER(TRIM(street_line_2)), ''), "
          "COALESCE(UPPER(TRIM(city)), ''), COALESCE(UPPER(TRIM(state)), ''), COALESCE(UPPER(TRIM(zip_code)), ''), "
          "COALESCE(UPPER(TRIM(occupation)), ''), COALESCE(UPPER(TRIM(employer)), '') "
          "FROM raw_table")

print("Done.")
conn.commit()

creating accounts table...
table created, inserting records
Done.


In [11]:
print('creating indexes on accounts table...')
c.execute("CREATE INDEX accounts_account_info ON accounts "
          "(name, address_1, address_2, city, "
          " state, zip)")
print("Done.")
conn.commit()

creating indexes on accounts table...
Done.


In [12]:
conn.commit()

In [13]:
print('creating committees table...')
c.execute("CREATE TABLE committees "
          "(comm_id SERIAL PRIMARY KEY, "
          " sboe_committee_id VARCHAR(200), name VARCHAR(200), "
          " address_1 VARCHAR(200), address_2 VARCHAR(200), "
          " city VARCHAR(200), state VARCHAR(200), zip VARCHAR(200), "
          " candidate_id INT, treasurer_id INT, asst_treasurer_id INT "
          ")")
print("inserting records")
c.execute("INSERT INTO committees "
          "(sboe_committee_id, name, address_1, address_2, city, state, zip) "
          "SELECT DISTINCT COALESCE(UPPER(TRIM(committee_sboe_id)), ''), "
          "COALESCE(UPPER(TRIM(committee_name)), ''), COALESCE(UPPER(TRIM(committee_street_1)), ''), COALESCE(UPPER(TRIM(committee_street_2)), ''), "
          "COALESCE(UPPER(TRIM(committee_city)), ''), COALESCE(UPPER(TRIM(committee_state)), ''), COALESCE(UPPER(TRIM(committee_zip_code)), '') "
          "FROM raw_table")

print("Done.")
conn.commit()

creating committees table...
inserting records
Done.


In [14]:
print('creating transactions table...')
c.execute("CREATE TABLE transactions "
                "(transaction_id INT, original_committee_sboe_id VARCHAR(200), original_account_id INT,  "
                " transaction_type VARCHAR(200), date_occured DATE, amount FLOAT, "
                " report_name VARCHAR(200), account_code VARCHAR(200), form_of_payment VARCHAR(200), "
                " purpose VARCHAR(500), candidate_referendum_name VARCHAR(200), declaration VARCHAR(200), "
                "canon_account_id INT, canon_committee_sboe_id VARCHAR(200) "
                ")")
print("Done.")
conn.commit()

creating transactions table...
Done.


In [15]:
print("Inserting transaction records")
c.execute("INSERT INTO transactions "
          "(transaction_id, original_committee_sboe_id, original_account_id, "
          " transaction_type, date_occured, amount, report_name, account_code, "
          " form_of_payment, purpose, candidate_referendum_name, declaration)"
          "SELECT trans_id, "
          "COALESCE(UPPER(TRIM(committee_sboe_id)), ''), "
          "accounts.account_id, "
          "COALESCE(UPPER(TRIM(transction_type)), ''), "
          "TO_DATE(TRIM(date_occured), 'MM/DD/YYYY'), "
          "CAST(amount as DOUBLE PRECISION), "
          "COALESCE(UPPER(TRIM(report_name)), ''), "
          "COALESCE(UPPER(TRIM(account_code)), ''), "
          "COALESCE(UPPER(TRIM(form_of_payment)), ''), "
          "COALESCE(UPPER(TRIM(purpose)), ''), "
          "COALESCE(UPPER(TRIM(candidate_referendum_name)), ''), "
          "COALESCE(UPPER(TRIM(declaration)), '') "
          "FROM raw_table LEFT JOIN accounts ON "
          "accounts.name = COALESCE(UPPER(TRIM(raw_table.name)), '') AND "
          "accounts.address_1 = COALESCE(UPPER(TRIM(raw_table.street_line_1)), '') AND "
          "accounts.address_2 = COALESCE(UPPER(TRIM(raw_table.street_line_2)), '') AND "
          "accounts.city = COALESCE(UPPER(TRIM(raw_table.city)), '') AND "
          "accounts.state = COALESCE(UPPER(TRIM(raw_table.state)), '') AND "
          "accounts.employer = COALESCE(UPPER(TRIM(raw_table.employer)), '') AND "
          "accounts.occupation = COALESCE(UPPER(TRIM(raw_table.occupation)), '') AND "
          "accounts.zip = COALESCE(UPPER(TRIM(raw_table.zip_code)), '')")
print("Done.")
conn.commit()

Inserting transaction records
Done.


In [16]:
print('creating indexes on transactions...')
c.execute("ALTER TABLE transactions ADD PRIMARY KEY(transaction_id)")
c.execute("CREATE INDEX donor_idx ON transactions (original_account_id)")
c.execute("CREATE INDEX committee_idx ON transactions (original_committee_sboe_id)")
c.execute("CREATE INDEX canon_donor_idx ON transactions (canon_account_id)")
print("Done.")
conn.commit()

creating indexes on transactions...
Done.


In [17]:
print('creating processed_accounts...')
c.execute("CREATE TABLE processed_accounts AS "
          "(SELECT account_id, "
          " CASE WHEN (name = '') "
          "      THEN NULL "
          "      ELSE LOWER(CONCAT_WS(' ', name)) "
          " END AS name, "  
           " CASE WHEN (address_1 = '' AND address_2 = '') "
          "      THEN NULL "
          "      ELSE LOWER(CONCAT_WS(' ', address_1, address_2)) "
          " END AS address, " 
          " CASE WHEN (city = '') "
          "      THEN NULL "
          "      ELSE LOWER(city) "
          " END AS city, "
          " CASE WHEN (state = '') "
          "      THEN NULL "
          "      ELSE LOWER(state) "
          " END AS state, "
          " CASE WHEN (zip = '') "
          "      THEN NULL "
          "      ELSE LOWER(zip) "
          " END AS zip, "
          " CASE WHEN (occupation = '') "
          "      THEN NULL "
          "      ELSE LOWER(occupation) " 
          " END AS occupation, "
          " CASE WHEN (employer = '') "
          "      THEN NULL "
          "      ELSE LOWER(employer) "
          " END AS employer "
        " FROM accounts)")

c.execute("CREATE INDEX processed_account_idx ON processed_accounts (account_id)")
print("Done.")
conn.commit()

creating processed_accounts...
Done.


In [18]:
print('creating processed_committees...')
c.execute("CREATE TABLE processed_committees AS "
          "(SELECT comm_id, "
          " CASE WHEN (name = '') "
          "      THEN NULL "
          "      ELSE LOWER(CONCAT_WS(' ', name)) "
          " END AS name, "  
           " CASE WHEN (address_1 = '' AND address_2 = '') "
          "      THEN NULL "
          "      ELSE LOWER(CONCAT_WS(' ', address_1, address_2)) "
          " END AS address, " 
          " LOWER(city) AS city, "
          " LOWER(state) AS state, "
          " LOWER(zip) AS zip "
          " FROM committees)")

c.execute("CREATE INDEX processed_committee_idx ON processed_committees (comm_id)")
print("Done.")
conn.commit()

creating processed_committees...
Done.


## **Transaction Views**
### Needs to be done after Dedupe Process

In [ ]:
conn = psycopg2.connect(database="camp-fin",
                        user="postgres",
                       password="",
                        host="172.16.238.13",
                        port="5432")
c = conn.cursor()

In [ ]:
conn.commit()

In [ ]:
print("Creating Contributions View")
c.execute("CREATE VIEW contributions AS "
          "(SELECT transactions.transaction_id AS trans_id, "
          " transactions.original_committee_sboe_id AS original_receiver_id, "
          " transactions.original_account_id AS original_source_id, "
          " transactions.transaction_type, "
          " 'C' AS transaction_category, "
          " transactions.date_occured, "
          " transactions.amount, "
          " transactions.report_name, "
          " transactions.account_code, "
          " transactions.form_of_payment, "
          " transactions.purpose, "
          " transactions.candidate_referendum_name AS candidate_refereendum_name, "
          " transactions.declaration, "
          " transactions.canon_account_id AS canon_source_id, "
          " transactions.canon_committee_sboe_id AS canon_receiver_id "
          "FROM transactions "
          " WHERE transactions.transaction_type IN ('CONTRIB TO REIMBURSE', 'DONATION', 'FORGIVEN LOAN', "
          " 'GENERAL', 'INDIVIDUAL', 'INTEREST', 'LOAN', "
          " 'OUTSIDE SOURCE', 'OUTSTANDING LOAN', 'NON-PARTY COMM', 'NONPROFIT', 'PARTY COMM'))")     
conn.commit()
print("Done.")

In [ ]:
print("Creating Expenses View")
c.execute("CREATE VIEW expenses AS "
          "SELECT transactions.transaction_id AS trans_id, "
          " transactions.original_committee_sboe_id AS orginal_source_id, "
          " transactions.original_account_id AS original_receiver_id, "
          " transactions.transaction_type, "
          " 'E' AS transaction_category, "
          " transactions.date_occured, "
          " transactions.amount, "
          " transactions.report_name, "
          " transactions.account_code, "
          " transactions.form_of_payment, "
          " transactions.purpose, "
          " transactions.candidate_referendum_name AS candidate_refereendum_name, "
          " transactions.declaration, "
          " transactions.canon_committee_sboe_id AS canon_source_id, "
          " transactions.canon_account_id AS canon_receiver_id "
          "FROM transactions "
          "WHERE transactions.transaction_type IN ('CONT TO OTHER COMM', 'COORD PARTY EXP', 'DEBT PAYMENT', "
          " 'INDEPENDENT EXP', 'LOAN REPAYMENT', 'NONMONETARY GIFT', 'OPERATING EXP', "
          " 'REFUND')")
conn.commit()
print("Done.")

In [ ]:
c.execute("CREATE TABLE processed_committees AS "
          "(SELECT comm_id, "
          " CASE WHEN (name = '') "
          "      THEN NULL "
          "      ELSE LOWER(CONCAT_WS(' ', name)) "
          " END AS name, "  
           " CASE WHEN (address_1 = '' AND address_2 = '') "
          "      THEN NULL "
          "      ELSE LOWER(CONCAT_WS(' ', address_1, address_2)) "
          " END AS address, " 
          " LOWER(city) AS city, "
          " LOWER(state) AS state, "
          " LOWER(zip) AS zip "
          " FROM committees)")

In [ ]:
c.close()
conn.close()
print('Done.')